## Introduction

Through this notebook, we are performing data analysis on a prescription drug data set from the British NHS using python. We'll attempt to identify what medical practices prescribe opioids at an usually high rate and what practices are prescribing substantially more rare drugs compared to the rest of the medical practices. We'll also try to utilize z-scores to help identify the aforementioned practices.

## Downloading the data

We first need to download the data we'll be using from Amazon S3:

In [1]:
!mkdir dw-data
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/201701scripts_sample.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/201606scripts_sample.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/practices.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/chem.csv.gz -nc -P ./dw-data/

--2021-12-26 23:20:39--  http://dataincubator-wqu.s3.amazonaws.com/dwdata/201701scripts_sample.csv.gz
Resolving dataincubator-wqu.s3.amazonaws.com (dataincubator-wqu.s3.amazonaws.com)... 52.217.96.140
Connecting to dataincubator-wqu.s3.amazonaws.com (dataincubator-wqu.s3.amazonaws.com)|52.217.96.140|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19011654 (18M) [text/csv]
Saving to: ‘./dw-data/201701scripts_sample.csv.gz’

201701scripts_sampl 100%[===================>]  18.13M  1.14MB/s    in 17s     

2021-12-26 23:20:56 (1.10 MB/s) - ‘./dw-data/201701scripts_sample.csv.gz’ saved [19011654/19011654]

--2021-12-26 23:20:56--  http://dataincubator-wqu.s3.amazonaws.com/dwdata/201606scripts_sample.csv.gz
Resolving dataincubator-wqu.s3.amazonaws.com (dataincubator-wqu.s3.amazonaws.com)... 54.231.200.41
Connecting to dataincubator-wqu.s3.amazonaws.com (dataincubator-wqu.s3.amazonaws.com)|54.231.200.41|:80... connected.
HTTP request sent, awaiting response... 200 OK


## Loading the data

In [2]:
import pandas as pd
import numpy as np
import gzip

In [3]:
# load the 2017 data
scripts = pd.read_csv("./dw-data/201701scripts_sample.csv.gz", compression = 'gzip')
scripts.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6


In [4]:
col_names = [ 'code', 'name', 'addr_1', 'addr_2', 'borough', 'village', 'post_code']
practices = pd.read_csv("./dw-data/practices.csv.gz", names = col_names, compression = 'gzip')
practices.head()

,code,name,addr_1,addr_2,borough,village,post_code
0,A81001,THE DENSHAM SURGERY,THE HEALTH CENTRE,LAWSON STREET,STOCKTON ON TEES,CLEVELAND,TS18 1HU
1,A81002,QUEENS PARK MEDICAL CENTRE,QUEENS PARK MEDICAL CTR,FARRER STREET,STOCKTON ON TEES,CLEVELAND,TS18 2AW
2,A81003,VICTORIA MEDICAL PRACTICE,THE HEALTH CENTRE,VICTORIA ROAD,HARTLEPOOL,CLEVELAND,TS26 8DB
3,A81004,WOODLANDS ROAD SURGERY,6 WOODLANDS ROAD,NaN,MIDDLESBROUGH,CLEVELAND,TS1 3BE
4,A81005,SPRINGWOOD SURGERY,SPRINGWOOD SURGERY,RECTORY LANE,GUISBOROUGH,NaN,TS14 7DJ


In [21]:
chem = pd.read_csv("./dw-data/chem.csv.gz", compression = 'gzip')
chem.head()

,CHEM SUB,NAME
0,0101010A0,Alexitol Sodium
1,0101010B0,Almasilate
2,0101010C0,Aluminium Hydroxide
3,0101010D0,Aluminium Hydroxide With Magnesium
4,0101010E0,Hydrotalcite


## Summary statistics

In [5]:
scripts.describe()

,items,nic,act_cost,quantity
count,973193.000000,973193.000000,973193.000000,973193.000000
mean,9.133136,73.058915,67.986613,741.329835
std,29.204198,188.070257,174.401703,3665.426958
min,1.000000,0.000000,0.040000,0.000000
25%,1.000000,7.800000,7.330000,28.000000
50%,2.000000,22.640000,21.220000,100.000000
75%,6.000000,65.000000,60.670000,350.000000
max,2384.000000,16320.000000,15108.320000,577720.000000


In [6]:
names = scripts.columns.to_list()[3:]
stats_list = [scripts.describe()[name].to_list() for name in names]

In [7]:
summary_stats = [(col, tuple([stats[0]*stats[1]] + stats[1:3] + stats[4:7])) for col, stats in zip(names, stats_list)]

In [8]:
summary_stats

[('items', (8888304.0, 9.133135976111625, 29.20419828297713, 1.0, 2.0, 6.0)),
 ('nic',
  (71100424.84000827,
   73.05891517921756,
   188.07025690683025,
   7.8,
   22.64,
   65.0)),
 ('act_cost',
  (66164096.11999956,
   67.98661326170611,
   174.40170332300627,
   7.33,
   21.22,
   60.67)),
 ('quantity',
  (721457006.0, 741.3298348837282, 3665.426958468499, 28.0, 100.0, 350.0))]

## Finding the most common item


In [9]:
bnf_items = scripts.groupby('bnf_name')['bnf_name', 'items'].sum()
most_common_item = [tuple(bnf_items.sort_values(by='items', ascending=False).reset_index().iloc[0])]

/tmp/ipykernel_52763/1901049707.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  bnf_items = scripts.groupby('bnf_name')['bnf_name', 'items'].sum()


In [10]:
most_common_item

[('Omeprazole_Cap E/C 20mg', 218583)]

## Classigying items by region

Now let's find the most common item by post code. The post code information is in the `practices` DataFrame, and we'll need to `merge` it into the `scripts` DataFrame.

In [11]:
post_codes_per_practice = practices.groupby('code').agg({'post_code': [list, 'count']}).sort_values(by = ('post_code', 'count'), ascending=False)
post_codes_per_practice.head(6)

post_code      
                                            list count
code                                                  
Y04804  [TR15 3ER, TR15 3ER, TR15 3ER, TR15 3ER]     4
F81223      [SS9 5UU, SS9 5UU, SS9 5UU, SS9 5UU]     4
J82024  [SO14 0YG, SO14 0YG, SO16 4XE, SO16 4XE]     4
D81631      [PE1 3BF, PE1 3BF, PE1 3BF, PE1 3BF]     4
B85060      [HD1 5PX, HD1 5PX, HD1 5PX, HD1 5PX]     4
Y04108   [SW19 1RH, SW1E 6QP, SW1E 6QP, SW6 4UL]     4

In [12]:
first_post_per_practice = pd.DataFrame(post_codes_per_practice[('post_code', 'list')].apply(lambda row : sorted(row)[0]).rename('post_code'))
first_post_per_practice.head()

,post_code
code,
Y04804,TR15 3ER
F81223,SS9 5UU
J82024,SO14 0YG
D81631,PE1 3BF
B85060,HD1 5PX


In [13]:
joined = pd.merge(scripts, first_post_per_practice, left_on='practice', right_on='code')[['practice', 'bnf_name', 'items', 'post_code']]
joined.head()

,practice,bnf_name,items,post_code
0,N85639,Bisacodyl_Tab E/C 5mg,1,CH44 5UF
1,N85639,Movicol Plain_Paed Pdr Sach 6.9g,1,CH44 5UF
2,N85639,Salbutamol_Inha 100mcg (200 D) CFF,1,CH44 5UF
3,N85639,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,CH44 5UF
4,N85639,Diazepam_Tab 2mg,1,CH44 5UF


In [14]:
items_per_post = joined.groupby(['post_code','bnf_name']).sum().sort_values(by='items', ascending=False).reset_index()
items_per_post = items_per_post.groupby('post_code').agg({'bnf_name': list, 'items':list})
items_per_post.head()

,bnf_name,items
post_code,,
B11 4BW,"[Salbutamol_Inha 100mcg (200 D) CFF, Paracet_T...","[706, 451, 387, 385, 350, 283, 249, 224, 211, ..."
B12 9LP,"[Paracet_Tab 500mg, Lansoprazole_Cap 30mg (E/C...","[425, 379, 366, 339, 261, 216, 212, 202, 201, ..."
B18 7AL,"[Salbutamol_Inha 100mcg (200 D) CFF, Metformin...","[556, 421, 416, 404, 348, 302, 268, 256, 242, ..."
B21 9RY,"[Metformin HCl_Tab 500mg, Salbutamol_Inha 100m...","[1033, 706, 675, 642, 596, 560, 515, 507, 430,..."
B23 6DJ,"[Lansoprazole_Cap 30mg (E/C Gran), Aspirin Dis...","[599, 449, 434, 415, 368, 360, 340, 337, 263, ..."


In [15]:
most_common_per_post = items_per_post.reset_index()[['post_code','bnf_name','items']].apply(lambda row : (row[0], row[1][0], row[2][0]/sum(row[2])), axis=1)
most_common_per_post.head()

0    (B11 4BW, Salbutamol_Inha 100mcg (200 D) CFF, ...
1    (B12 9LP, Paracet_Tab 500mg, 0.02489310607391788)
2    (B18 7AL, Salbutamol_Inha 100mcg (200 D) CFF, ...
3    (B21 9RY, Metformin HCl_Tab 500mg, 0.033293583...
4    (B23 6DJ, Lansoprazole_Cap 30mg (E/C Gran), 0....
dtype: object

In [16]:
items_by_region = most_common_per_post.to_list()[:100]

In [17]:
items_by_region

[('B11 4BW', 'Salbutamol_Inha 100mcg (200 D) CFF', 0.031058906339360346),
 ('B12 9LP', 'Paracet_Tab 500mg', 0.02489310607391788),
 ('B18 7AL', 'Salbutamol_Inha 100mcg (200 D) CFF', 0.027111371172225472),
 ('B21 9RY', 'Metformin HCl_Tab 500mg', 0.03329358300834757),
 ('B23 6DJ', 'Lansoprazole_Cap 30mg (E/C Gran)', 0.021384456106529576),
 ('B61 0AZ', 'Omeprazole_Cap E/C 20mg', 0.028713318284424378),
 ('B70 7AW', 'Paracet_Tab 500mg', 0.025135992162726547),
 ('B72 1RL', 'Omeprazole_Cap E/C 20mg', 0.020228765092141495),
 ('B8 1RZ', 'Metformin HCl_Tab 500mg', 0.021347750961484866),
 ('B9 5PU', 'Ventolin_Evohaler 100mcg (200 D)', 0.024826024522257815),
 ('B90 3LX', 'Omeprazole_Cap E/C 20mg', 0.026965103983080718),
 ('BA5 1XJ', 'Omeprazole_Cap E/C 20mg', 0.028261290947858113),
 ('BB11 2DL', 'Omeprazole_Cap E/C 20mg', 0.027381741821396993),
 ('BB2 1AX', 'Omeprazole_Cap E/C 20mg', 0.03428191046763188),
 ('BB3 1PY', 'Omeprazole_Cap E/C 20mg', 0.032683395995453356),
 ('BB4 5SL', 'Omeprazole_Cap E/

In [18]:
#most_common_post = joined.groupby(['post_code','bnf_name']).sum().sort_values(by='items', ascending=False).reset_index().groupby('post_code').head(1)

#most_common_post = most_common_post.set_index('post_code').sort_index()

#total_items = joined.groupby('post_code').sum()

#most_common_post[['proportion_per_post']] = most_common_post[['items']]/total_items

#items_by_region = list([(index, row['bnf_name'], row['proportion_per_post']) for index, row in most_common_post.iterrows()])[:00]

## Identifying script anomalies

Drug abuse is a source of human and monetary costs in health care. A first step in identifying practitioners that enable drug abuse is to look for practices where commonly abused drugs are prescribed unusually often. Let's try to find practices that prescribe an unusually high amount of opioids. The opioids we'll look for are given in the list below.

In [19]:
opioids = ['morphine', 'oxycodone', 'methadone', 'fentanyl', 'pethidine', 'buprenorphine', 'propoxyphene', 'codeine']

These are generic names for drugs, not brand names. Generic drug names can be found using the `'bnf_code'` field in `scripts` along with the `chem` table.. Use the list of opioids provided above along with these fields to make a new field in the `scripts` data that flags whether the row corresponds with a opioid prescription.

In [22]:
#Detect drug brand names that contain opioids with 'is_opioid'
chem['is_opioid'] = chem['NAME'].str.contains('|'.join(opioids), case=False).astype(int)

#Flag scripts that correspond to an opioid prescription by adding 'is_opioid' column to scripts
scripts = pd.merge(scripts, chem, left_on='bnf_code', right_on='CHEM SUB', how='left')[scripts.columns.tolist()+['is_opioid']]
scripts['is_opioid'] = scripts['is_opioid'].fillna(0).astype(int)

In [23]:
scripts.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity,is_opioid
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12,0
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30,0
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1,0
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150,0
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6,0


In [24]:
opioids_per_practice = scripts.groupby('practice')['is_opioid'].mean()

In [25]:
relative_opioids_per_practice = opioids_per_practice - scripts['is_opioid'].mean() 

Now that we know the difference between each practice's opioid prescription rate and the overall rate, we can identify which practices prescribe opioids at above average or below average rates. However, are the differences from the overall rate important or just random deviations? In other words, are the differences from the overall rate big or small?

To answer this question we have to quantify the difference we would typically expect between a given practice's opioid prescription rate and the overall rate. This quantity is called the **standard error**, and is related to the **standard deviation**, $\sigma$. The standard error in this case is

$$ \frac{\sigma}{\sqrt{n}} $$

where $n$ is the number of prescriptions each practice made. Calculate the standard error for each practice. Then divide `relative_opioids_per_practice` by the standard errors. We'll call the final result `opioid_scores`.

In [26]:
standard_error_per_practice = scripts['is_opioid'].std()/np.sqrt(scripts.groupby('practice')['is_opioid'].count())

opioid_scores = (relative_opioids_per_practice/standard_error_per_practice).rename('opioid_score')

In [27]:
opioid_scores.head()

practice
A81005   -0.551807
A81007    1.540186
A81011    2.293528
A81012    1.373723
A81017    0.586611
Name: opioid_score, dtype: float64

In [28]:
#We can notice that some codes are associated with multiple names
practices.groupby('code')['name'].nunique().sort_values(ascending=False).head()

code
N82646    3
Y04804    3
Y04565    3
F85065    3
P92021    3
Name: name, dtype: int64

In [29]:
#Retrieve the first alphabetically name for each practice code
practice_names = practices.groupby('code')['name'].apply(lambda group: sorted([row for row in group])[0])

In [30]:
#Compute the number of scripts for each practice
scripts_per_practice = scripts.groupby('practice').size().rename('scripts_number')

In [31]:
#Combine each practice name with its opioid score and number of scripts
practice_results = pd.concat([practice_names, opioid_scores, scripts_per_practice], axis=1, join='inner')
practice_results.head()

,name,opioid_score,scripts_number
A81005,SPRINGWOOD SURGERY,-0.551807,1509
A81007,BANKHOUSE SURGERY,1.540186,1456
A81011,CHADWICK PRACTICE,2.293528,1569
A81012,WESTBOURNE MEDICAL CENTRE,1.373723,1333
A81017,WOODBRIDGE PRACTICE,0.586611,2151


The quantity we have calculated in `opioid_scores` is called a **z-score**:

$$ \frac{\bar{X} - \mu}{\sqrt{\sigma^2/n}} $$

Here $\bar{X}$ corresponds with the proportion for each practice, $\mu$ corresponds with the proportion across all practices, $\sigma^2$ corresponds with the variance of the proportion across all practices, and $n$ is the number of prescriptions made by each practice. Notice $\bar{X}$ and $n$ will be different for each practice, while $\mu$ and $\sigma$ are determined across all prescriptions, and so are the same for every z-score. The z-score is a useful statistical tool used for hypothesis testing, finding outliers, and comparing data about different types of objects or events.


In [32]:
anomalies = sorted([(row[1],row[2],row[3]) for row in practice_results.itertuples()], key=lambda x:x[1], reverse=True)[:100]  
#anomalies = [("NATIONAL ENHANCED SERVICE", 11.6958178629, 7)] * 100

In [33]:
anomalies[:5]

[('NATIONAL ENHANCED SERVICE', 11.700971787531765, 7),
 ('OUTREACH SERVICE NH / RH', 7.342237296128809, 2),
 ('BRISDOC HEALTHCARE SERVICES OOH', 6.154319628897654, 60),
 ('H&R P C SPECIAL SCHEME', 5.126072754033091, 36),
 ('HMR BARDOC OOH', 4.963767331533176, 321)]

## Identifying script growth

Another way to identify anomalies is by comparing current data to historical data. In the case of identifying sites of drug abuse, we might compare a practice's current rate of opioid prescription to their rate 5 or 10 years ago. Unless the nature of the practice has changed, the profile of drugs they prescribe should be relatively stable. We might also want to identify trends through time for business reasons, identifying drugs that are gaining market share.

In [34]:
scripts16 = pd.read_csv('./dw-data/201606scripts_sample.csv.gz', compression='gzip')

In [35]:
scripts16.head(3)

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity
0,N85638,0301011R0,Salamol_Inha 100mcg (200 D) CFF (Teva),2,2.92,2.73,2
1,N85638,0301011R0,Easyhaler_Salbutamol Sulf 200mcg (200D),1,6.63,6.15,1
2,N85638,0301020I0,Ipratrop Brom_Inh Soln 500mcg/2ml Ud,1,1.77,1.75,12


In [36]:
#Compute both 2016 and 2017 prescription count for each bnf_name
bnf16 = scripts16.groupby('bnf_name').size().rename('bnf16_count')
bnf17 = scripts.groupby('bnf_name').size().rename('bnf17_count')

In [37]:
#Merge bnf16 and bnf17 data and eliminate occurences with no prescriptions in 2016 or in 2017
bnf = pd.concat([bnf16, bnf17], axis=1, sort=True).dropna(axis=0, how='any')

#Compute the percent growth in prescription rate from 2016 to 2017
bnf['growth_rate'] = (bnf['bnf17_count']-bnf['bnf16_count'])/bnf['bnf16_count']

In [38]:
#Filter out drugs items with less than 50 prescriptions in 2016 and sort items by growth rate
bnf_high_count = bnf[bnf['bnf16_count'] >= 50].sort_values(by='growth_rate', ascending=False)

In [39]:
bnf_high_count.head(3)

,bnf16_count,bnf17_count,growth_rate
bnf_name,,,
Butec_Transdermal Patch 5mcg/hr,62.0,277.0,3.467742
Butec_Transdermal Patch 10mcg/hr,69.0,276.0,3.000000
Fostair NEXThaler_Inh 200mcg/6mcg (120D),86.0,209.0,1.430233


In [40]:
#Retrieve 50 items with largest growth and 50 items with largest shrinkage (high negative growth)
largest_growth = bnf_high_count.head(50)
largest_shrinkage = bnf_high_count.tail(50)

#Combine results into a new DataFrame
growth_anomalies = pd.concat([largest_growth, largest_shrinkage])

In [41]:
#Retrieve results as a list of tuples in format ('bnf_name', 'growth_rate', 'bnf16_count')
script_growth = [(row[0], row[3], row[1]) for row in growth_anomalies.itertuples()]

In [42]:
script_growth

[('Butec_Transdermal Patch 5mcg/hr', 3.467741935483871, 62.0),
 ('Butec_Transdermal Patch 10mcg/hr', 3.0, 69.0),
 ('Fostair NEXThaler_Inh 200mcg/6mcg (120D)', 1.430232558139535, 86.0),
 ('Sytron_Elix 190mg/5ml S/F', 1.280130293159609, 307.0),
 ('Pneumococcal_Vac 0.5ml Vl (23 Valent)', 1.2694300518134716, 193.0),
 ('Spiolto Respimat_Inha2.5/2.5mcg(60D)+Dev', 1.2692307692307692, 52.0),
 ('Trulicity_Inj 1.5mg/0.5ml Pf Pen', 1.1851851851851851, 54.0),
 ('CosmoCol_Paed Oral Pdr Sach 6.9g', 1.1774193548387097, 62.0),
 ('Dulaglutide_Inj 1.5mg/0.5ml Pf Dev', 0.9634146341463414, 82.0),
 ('Sod Feredetate_Oral Soln 190mg/5ml S/F', 0.9181818181818182, 440.0),
 ('ViATIM_Vac D/Chamber 160u/25mcg 1ml Pfs', 0.912, 125.0),
 ('Empagliflozin_Tab 25mg', 0.896, 125.0),
 ('CareSens Lancets 0.31mm/30 Gauge', 0.8955223880597015, 67.0),
 ('Fostair_Inh 200mcg/6mcg (120D) CFF', 0.8616600790513834, 253.0),
 ('Orbis Nor Saline Sod Chlor 0.9% Nsl Dps', 0.8588235294117647, 85.0),
 ('Umeclidinium Brom_Inh 65mcg (30D)

## Finding rare scripts

Does a practice's prescription costs originate from routine care or from reliance on rarely prescribed treatments? Commonplace treatments can carry lower costs than rare treatments because of efficiencies in large-scale production. While some specialist practices can't help but avoid prescribing rare medicines because there are no alternatives, some practices may be prescribing a unnecessary amount of brand-name products when generics are available. Let's identify practices whose costs disproportionately originate from rarely prescribed items.


In [43]:
scripts = pd.read_csv("./dw-data/201701scripts_sample.csv.gz", compression = 'gzip')

In [44]:
#Compute the uniform probability of randomly chosing a bnf_code from the options available in data
p = 1/scripts['bnf_code'].nunique()

#Compute the rates at which each bnf_code is prescribed
rates = (scripts.groupby('bnf_code').size()/len(scripts)).rename('prescription_rate')

#Find rare bnf_code
rare_codes = rates <= 0.1*p

#Flag scripts prescribing rare bnf_code
scripts.set_index('bnf_code', inplace=True)
scripts['rare'] = pd.DataFrame(rare_codes)

In [45]:
scripts.head(3)

,practice,bnf_name,items,nic,act_cost,quantity,rare
bnf_code,,,,,,,
0106020C0,N85639,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12,False
0106040M0,N85639,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30,False
0301011R0,N85639,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1,False


Now for each practice, calculate the proportion of costs that originate from prescription of rare treatments (i.e. rare `'bnf_code'`). Use the `'act_cost'` field for this calculation.

In [46]:

practice_rare_cost = scripts[scripts['rare']].groupby('practice')['act_cost'].sum()
practice_total_cost = scripts.groupby('practice')['act_cost'].sum()
practice_rare_cost_prop = (practice_rare_cost/practice_total_cost).fillna(0)

Now we will calculate a z-score for each practice based on this proportion.
First take the difference of `rare_cost_prop` and the proportion of costs originating from rare treatments across all practices.

In [47]:
total_rare_cost = scripts[scripts['rare']]['act_cost'].sum()
total_cost = scripts['act_cost'].sum()
mean_rare_cost_prop = total_rare_cost/total_cost

relative_rare_cost_prop = practice_rare_cost_prop - mean_rare_cost_prop

Now we will estimate the standard errors (i.e. the denominator of the z-score) by simply taking the standard deviation of this difference.

In [48]:
standard_error = relative_rare_cost_prop.std()

Finally compute the z-scores. Return the practices with the top 100 z-scores in the form `(post_code, practice_name, z-score)`. Note that some practice codes will correspond with multiple names. In this case, use the first match when sorting names alphabetically.

In [49]:
rare_scores = relative_rare_cost_prop/standard_error

In [50]:
practice_names = practices.groupby('code')['name'].apply(lambda group: sorted([row for row in group])[0])

In [51]:
practice_scores = pd.concat([rare_scores, practice_names], axis=1, join='inner').sort_values(by='act_cost', ascending=False)

In [53]:
practice_scores.head(3)

,act_cost,name
Y03472,16.262687,CONSULTANT DIABETES TEAM
Y05320,15.128648,DMC COMMUNITY DERMATOLOGY RBWF
Y04404,7.542139,OUTPATIENTS JUBILEE HEALTH CENTRE


In [54]:
#We'll look at the top 100 z-scores
rare_scripts = [(row[0], row[2], row[1]) for row in practice_scores.itertuples()][:100]

In [55]:
rare_scripts[:3]

[('Y03472', 'CONSULTANT DIABETES TEAM', 16.262687124655088),
 ('Y05320', 'DMC COMMUNITY DERMATOLOGY RBWF', 15.128648195416883),
 ('Y04404', 'OUTPATIENTS JUBILEE HEALTH CENTRE', 7.542139356104627)]

*Copyright &copy; 2020 The Data Incubator.  All rights reserved.*